<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/Glob_Analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://github.com/profteachkids/StemUnleashed/raw/main/glob_data.zip -O data.zip

In [2]:
import shutil
import pandas as pd
import numpy as np
from pathlib import Path
import itertools as it

In [3]:
shutil.unpack_archive('data.zip','data')

In [4]:
files = list(Path('data').glob('**/*.csv'))

In [5]:
df_list=[]
for afile in files:
    df=pd.read_csv(afile, index_col=False,header=None)
    df_list.append(df)

In [36]:
files_split=[afile.as_posix().split('/') for afile in files]
dates=pd.to_datetime([file_split[-1].split('.')[0] + file_split[-2] + file_split[1] for file_split in files_split])
names=[file_split[-3] for file_split in files_split]

In [38]:
keys=[(date,name) for date,name in zip(dates,names)]

In [44]:
df=pd.concat(df_list,keys=keys)
df.index=df.index.droplevel(2)
df.index.names=['date','name']

In [50]:
date_index=df.index.get_level_values(0)
groups=df.groupby([date_index.year,date_index.month,date_index.day,'name'])

In [52]:
def regress(df_day):
    return np.polyfit(df_day.loc[:,0],df_day.loc[:,1],1)

In [56]:
df_fit=groups.apply(regress)

In [58]:
df_fit.index.names=['year','month','day','name']

In [59]:
df_fit=pd.DataFrame(df_fit.to_list(),index=df_fit.index,columns=['m','b'])

In [68]:
summarize=pd.pivot_table(df_fit, values=['b','m'], index=['day'], columns='name', aggfunc=['mean'])

In [69]:
summarize=summarize.sort_index()

In [70]:
summarize.style.format(precision=1)